# NutriScan MY - YOLOv8 Training Notebook

## 马来西亚食物识别模型训练

**Project**: Malaysian Food Detection  
**Model**: YOLOv8  
**Target**: 20 food categories, 1000+ images  
**Goal**: mAP50 > 80%, inference < 100ms

---


## 1. Environment Setup


In [1]:
# Install dependencies
%pip install ultralytics roboflow opencv-python pillow -q

# Verify installation
import ultralytics
print(f"✅ Ultralytics version: {ultralytics.__version__}")

# Check GPU availability
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using device: {device}")
if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\ngiva\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



Note: you may need to restart the kernel to use updated packages.
✅ Ultralytics version: 8.3.204
✅ Using device: cpu


## 2. Download Dataset from Roboflow


In [2]:
from roboflow import Roboflow

# Initialize Roboflow
# TODO: Replace with your actual API key from https://app.roboflow.com/settings/api
rf = Roboflow(api_key="BwTemPbP39LHLFH4teds")

# Download dataset
project = rf.workspace("malaysian-food-detection").project("malaysian-food-detection-wy3kt")
dataset = project.version(1).download("yolov8")

print(f"✅ Dataset downloaded to: {dataset.location}")


loading Roboflow workspace...
loading Roboflow project...


RuntimeError: Version number 1 is not found.

## 3. Verify Dataset Structure


In [ ]:
import os
from pathlib import Path

# Check dataset structure
dataset_path = Path(dataset.location)
data_yaml = dataset_path / "data.yaml"

print("📂 Dataset structure:")
print(f"   Train images: {len(list((dataset_path / 'train/images').glob('*')))}")
print(f"   Valid images: {len(list((dataset_path / 'valid/images').glob('*')))}")
print(f"   Test images: {len(list((dataset_path / 'test/images').glob('*')))}")

# Read data.yaml
import yaml
with open(data_yaml, 'r') as f:
    data_config = yaml.safe_load(f)
    
print(f"\n📊 Classes: {data_config['nc']}")
print(f"   {data_config['names']}")


## 4. Train YOLOv8 Model


In [ ]:
from ultralytics import YOLO

# Load pretrained model
model = YOLO('yolov8n.pt')  # nano model for mobile deployment

# Training configuration
results = model.train(
    data=str(data_yaml),
    epochs=100,              # 可根据需要调整
    imgsz=640,               # 输入图像大小
    batch=16,                # batch size (根据GPU调整)
    device=device,
    project='../../results',
    name='nutriscan_v1',
    
    # Hyperparameters
    lr0=0.01,                # initial learning rate
    lrf=0.01,                # final learning rate
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    
    # Augmentation
    hsv_h=0.015,             # HSV-Hue augmentation
    hsv_s=0.7,               # HSV-Saturation augmentation
    hsv_v=0.4,               # HSV-Value augmentation
    degrees=0.0,             # rotation
    translate=0.1,           # translation
    scale=0.5,               # scaling
    fliplr=0.5,              # flip left-right
    mosaic=1.0,              # mosaic augmentation
    
    # Early stopping
    patience=50,             # epochs to wait for improvement
    
    # Logging
    save=True,
    verbose=True,
    plots=True
)

print("\n✅ Training completed!")


## 5. Evaluate Model Performance


In [ ]:
# Evaluate on validation set
metrics = model.val()

print("📊 Performance Metrics:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.p:.3f}")
print(f"   Recall: {metrics.box.r:.3f}")

# Check if target is met
if metrics.box.map50 > 0.80:
    print("\n✅ Target achieved! mAP50 > 80%")
else:
    print(f"\n⚠️  Need improvement. Current: {metrics.box.map50:.1%}, Target: 80%")


## 6. Export to TensorFlow Lite


In [ ]:
# Export to TFLite format for mobile deployment
tflite_model = model.export(format='tflite', imgsz=640, int8=False)

print(f"✅ TFLite model exported: {tflite_model}")

# Check model size
import os
model_size = os.path.getsize(tflite_model) / (1024 * 1024)  # MB
print(f"📦 Model size: {model_size:.2f} MB")

if model_size < 20:
    print("✅ Target achieved! Model < 20MB")
else:
    print(f"⚠️  Model too large. Current: {model_size:.1f}MB, Target: < 20MB")

# Save to models directory
import shutil
output_dir = Path('../../models/nutriscan_v1')
output_dir.mkdir(parents=True, exist_ok=True)
shutil.copy(tflite_model, output_dir / 'model.tflite')
print(f"\n✅ Model saved to: {output_dir / 'model.tflite'}")


## Summary

### 完成项目
- ✅ 环境配置
- ✅ 数据集下载
- ✅ 模型训练
- ✅ 性能评估
- ✅ TFLite导出

### 下一步 (Week 5-6)
1. 将TFLite模型集成到React Native app
2. 实现相机接口和图像预处理
3. 集成Gemini Vision API进行营养分析
4. 开发多语言UI界面

---

**Created**: 2025-10-04  
**Model**: YOLOv8n  
**Dataset**: 20 Malaysian food categories
